## Set Up

In [ ]:
pip install arcgis

In [1]:
import arcpy
from datetime import datetime
import os
import sys
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
import sqlalchemy as sa
import pandas as pd
from arcgis import GIS
from arcgis.features import FeatureSet, GeoAccessor, GeoSeriesAccessor, FeatureLayer
import pandas as pd
import numpy as np
import requests
from arcgis.gis import GIS
from arcgis.geometry import SpatialReference
#import geopandas as gpd to use spatial.reference stuff
gis = GIS()
# # Set Pandas display options to show all rows and columns
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

# in memory output file path
wk_memory = "memory" + "\\"


# set workspace and sde connections 
#working_folder = "C:\GIS"

#workspace      = "C:\GIS\Scratch.gdb"
#arcpy.env.workspace = "C:\GIS\Scratch.gdb"
#workspace ='F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Scratch.gdb'
#arcpy.env.workspace = 'F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Scratch.gdb'

# network path to connection files
filePath = "F:\\GIS\\DB_CONNECT"

#path to GDB's to update and master data
master_path = "F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Data\SEZ_Data.gdb"
SEZ_Master = os.path.join(master_path, "AssessmentUnits_Master")
#set workspace for connection to GDB
#workspace=master_path
# database file paths 
### SDE Collection New data collected is put into SDE.Survey under the indicator name
### SDE Vector is where the data will go 
#sdeBase    = os.path.join(filePath, "Vector.sde")
sdeCollect = os.path.join(filePath, "Collection.sde")
sdeBase    = os.path.join(filePath, "SarahVector.sde")
#sdeCollect = os.path.join(filepath, "SarahCollect.sde")

# setup connection string???
connection_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=sql12;DATABASE=sde;UID=sde;PWD=staff"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(connection_url)

#with engine.begin() as sdeConnect:
 #   erosiondf      = pd.read_sql("SELECT * FROM sde.SDE.Stream_Erosion", sdeConnect)

# local variables sdata is starting data and f data is finishing datatables
ffdata = os.path.join(sdeBase, "sde.SDE.Monitoring")
sdata = os.path.join(sdeCollect, "sde.SDE.Survey")
fdata = os.path.join(master_path, "AssessmentUnits_Master")

##Tables we get the data from in Collect
sezsurveytable = os.path.join(sdata, "sde.SDE.sez_survey")
erosiondata = os.path.join(sdata, "sde.SDE.Stream_Erosion")
incisiondata = os.path.join(sdata, "sde.SDE.sez_channel_incision")
invasivedata = os.path.join(sdata, "sde.SDE.sez_invasive_plant")
headcutdata = os.path.join(sdata, "sde.SDE.sez_stream_headcut")

#make this a spatial df
streamdata = os.path.join(ffdata, "sde.SDE.Stream")


#Staging Tables currently living in SEZ_Data.GDB
stage_bank_stability = os.path.join(master_path, "bank_stability") 
stage_All_SEZ_Scores = os.path.join(master_path, "All_SEZ_Scores")
stage_biotic_integrity = os.path.join(master_path, "biotic_integrity")
stage_headcuts = os.path.join(master_path, "headcuts_table")
stage_incision = os.path.join(master_path, "incision")
stage_invasives = os.path.join(master_path, "invasives")



#Final table to append to
#finalSEZtable = os.path.join(fdata, "sde.SDE.SEZ_Assessment_Unit")
#finalSEZtable = os.path.join(master_path, "AssessmentUnits_Master")


# network path to connection files??????
#filePath = "C:\\GIS\\DB_CONNECT"
# network path to connection files
#filePath = r'F:\Research and Analysis\Workspace\Sarah'

#--------------------------------------------#
#Notes to self
#--------------------------------------------#
#F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Data\SEZ_Data.gdb. 
###'Assessment_Unit_Master' has all data for SEZ
###“Bioassessment Sample Locations” has stream sites in SEZs

gdbworking_folder = "F:\GIS\GIS_DATA\Monitoring"
headcutgdbfolder = os.path.join(gdbworking_folder, "Stream_Headcut", "StreamHeadcut_Survey")
invasivegdbfolder = os.path.join(gdbworking_folder, "Invasive_Species", "Invasive_Species_Survey")
## GDB with Raw Data straight from S123 not in the original folder (that one is not edited)
headcut23gdb = os.path.join(headcutgdbfolder, "Stream_Headcut_Survey_2023.gdb")
headcut22gdb = os.path.join(headcutgdbfolder, "Stream_Headcut_Survey_2022.gdb")
headcut20gdb = os.path.join(headcutgdbfolder, "Stream_Headcut_Survey_2020.gdb")
headcut19gdb = os.path.join(headcutgdbfolder, "Stream_Headcut_Survey_2019.gdb")
#erosion23gdb = os.path.join(gdbworking_folder, "Stream_Erosion","Stream_Erosion_Survey", "Stream_Erosion_Survey_2023.gdb")
#erosion22gdb = os.path.join(working_folder, "Stream_Erosion","Stream_Erosion_Survey", "Stream_Erosion_Survey_2022.gdb")
#erosion20gdb = os.path.join(working_folder, "Stream_Erosion","Stream_Erosion_Survey", "Stream_Erosion_Survey_2020.gdb")
#channelincision23gdb = os.path.join(working_folder,"Channel_Incision","Channel_Incision_Survey","Channel_Incision_Survey_2023.gdb")
#channelincision22gdb = os.path.join(working_folder, "Channel_Incision","Channel_Incision_Survey","Channel_Incision_Survey_2022.gdb")
#channelincision20gdb = os.path.join(working_folder, "Channel_Incision","Channel_Incision_Survey","Channel_Incision_Survey_2020.gdb")
invasiveplant23gdb= os.path.join(invasivegdbfolder,"Invasive_Species_Survey_2023.gdb")
invasiveplant22gdb= os.path.join(invasivegdbfolder,"Invasive_Species_Survey_2022.gdb")
invasiveplant20gdb= os.path.join(invasivegdbfolder,"Invasive_Species_Survey_2020.gdb")
invasiveplant19gdb= os.path.join(invasivegdbfolder,"Invasive_Species_Survey_2019.gdb")

#This is thelocatoin for the final SEZ GDB to be updated in the gdb on f drive in the AssessmentUnits Master (polygon) i believe
#FinalGDBtoupdate:F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Data \SEZ_Data.gdb

#Location of USFS Invasive Species Data
#https://apps.fs.usda.gov/arcx/rest/services/EDW/EDW_InvasiveSpecies_01/MapServer

#Monitoring Dashboard location
#Finalsdelocation:f'Vector.SDE' Sde.Monitoring Sde. SEZ_Assessment_Unit

#Threshold Location? sde.tabular

### Don't Use-Lookup Dictionary for SEZ ID's -Old code uses AssessmentUnit_Master feature class in SEZ_Data.gdb- don't use for now


In [ ]:
#-----------------------------------------------------------------------------------#
#Create look up dictionary for SEZ_ID fill in--currently works for stream erosion code--
#-----------------------------------------------------------------------------------#
#Add in SEZ_Survey table to match up parentglobalid to assessment unit name so we can match an sez_id
#----------------------------------------------------------------##Convert feature class in gdb to Pandas DataFrame this is for REST Service
# Set workspace environment
arcpy.env.workspace = master_path

# Specify the feature class name SEZ_Master
#feature_class = "AssessmentUnits_Master"
# Create a cursor to iterate over the rows in the feature class
fields = ['Assessment_Unit_Name', 'SEZ_ID']

data = []


# Iterate over the rows in the second feature class and append to data list
with arcpy.da.SearchCursor(SEZ_Master, fields) as cursor:
    for row in cursor:
        data.append(row)
# Convert the data to a Pandas DataFrame
df = pd.DataFrame(data, columns=fields)

# Define function to update SEZ ID lookup dictionary
def update_SEZID_lookup_dict(df, lookup_dict):
    for index, row in df.iterrows():
        # Update SEZ_ID column in DataFrame with data from the lookup dictionary
        df.at[index, 'SEZ_ID'] = lookup_dict.get(row['Assessment_Unit_Name'], None)

    return df

# Assuming 'Assessment_Unit_Name' is the common identifier between DataFrame and lookup dictionary
selected_columns = ['Assessment_Unit_Name', 'SEZ_ID']

# Drop duplicates based on 'Assessment_Unit_Name' and keep the first occurrence
unique_values = df.drop_duplicates(subset='Assessment_Unit_Name', keep='first')

# Convert selected columns to dictionary
lookup_dict = unique_values[selected_columns].set_index('Assessment_Unit_Name').to_dict(orient='index')

# Display the dictionary
print(lookup_dict)

# Update 'SEZ_ID' in other DataFrame with data from the lookup dictionary do this for each indicator...
#Other_df = update_SEZID_lookup_dict(Other_df, lookup_dict)


In [ ]:
#-----------------------------------------------------------------------------------#
# DONT USE YETT Create look up dictionary for SEZ_ID fill in with Global ID,SEZ ID, and Assessment Unit Name

# Set workspace environment
arcpy.env.workspace = master_path

# Define the fields for SEZ_Master and the additional feature class
Masterfields = ['Assessment_Unit_Name', 'SEZ_ID']
surveyfields = ['assessment_unit_name', 'GlobalID']  # Add any additional fields you need

# Create empty lists to store data
Masterdata = []
surveydata = []

# Iterate over the rows in the SEZ_Master feature class and append to data list
with arcpy.da.SearchCursor(SEZ_Master, Masterfields) as cursor:
    for row in cursor:
        Masterdata.append(row)

# Iterate over the rows in the additional feature class and append to data list
with arcpy.da.SearchCursor(sezsurveytable, surveyfields) as cursor:
    for row in cursor:
        surveydata.append(row)

# Convert the data to Pandas DataFrames
Masterdf = pd.DataFrame(Masterdata, columns=Masterfields)
surveydf = pd.DataFrame(surveydata, columns=surveyfields)
# Convert the column name in surveydf to match Masterdf
surveydf = surveydf.rename(columns={'assessment_unit_name': 'Assessment_Unit_Name'})
# Merge the DataFrames on 'Assessment_Unit_Name'
SEZIDdf = pd.merge(Masterdf, surveydf, on='Assessment_Unit_Name', how='inner')

# Check for missing values in 'GlobalID' or 'SEZ_ID'
missing_values = SEZIDdf[SEZIDdf['GlobalID'].isna() | SEZIDdf['SEZ_ID'].isna()]

# Replace missing values in 'SEZ_ID' with NaN
SEZIDdf.loc[missing_values.index, 'SEZ_ID'] = np.nan



# Display the merged DataFrame
print(SEZIDdf)
# Assuming 'Assessment_Unit_Name' is the common identifier between DataFrame and lookup dictionary
selected_columns = ['Assessment_Unit_Name', 'SEZ_ID', 'GlobalID']

# Drop duplicates based on 'Assessment_Unit_Name' and keep the first occurrence
unique_values = SEZIDdf.drop_duplicates(subset='Assessment_Unit_Name', keep='first')

# Convert selected columns to dictionary
lookup_dict = unique_values[selected_columns].set_index('Assessment_Unit_Name').to_dict(orient='index')

# Display the dictionary
print(lookup_dict)

### SEZ ID based off of excel lookup dictionary larger polygons only

In [2]:
#-----------------------------------------------------------------------------------#
#Create look up dictionary for SEZ_ID fill in--currently works for stream erosion code--- change to use excel so its not messy
#-----------------------------------------------------------------------------------#

# Step 1: Read the Excel file into a DataFrame
excel_data = pd.read_excel("F:\GIS\PROJECTS\ResearchAnalysis\SEZ\SEZID lookup.xlsx")  # Replace 'your_excel_file.xlsx' with the path to your Excel file

#Define Empty look up dataframe
lookup_dict = {}

for index, row in excel_data.iterrows():
    lookup_dict[row['Assessment_Unit_Name']] = row['SEZ_ID']

# See dictionary where keys are Assessment Unit Names and values are SEZ IDs
print(lookup_dict)




{'Angora Creek - tributary': 519, 'Angora Creek - upper': 520, 'Angora meadows - 1': 87, 'Angora meadows - 2': 90, 'Angora meadows - 3': 91, 'Angora meadows - 4': 143, 'Angora meadows - 5': 144, 'Angora meadows - 6': 142, 'Angora meadows - 7': 89, 'Angora meadows - 8': 88, 'Angora meadows - 9': 92, 'Angora meadows tributary - 1': 217, 'Angora meadows tributary - 2': 99, 'Angora meadows tributary - 3': 97, 'Angora meadows tributary - 4': 94, 'Angora meadows tributary - 5': 214, 'Angora meadows tributary - 6': 146, 'Angora meadows tributary - 7': 93, 'Angora meadows tributary - 8': 95, 'Angora meadows tributary - 9': 96, 'Angora tributary': 446, 'Antone meadows': 187, 'Baldwin marsh - 1': 160, 'Benwood meadows - 1': 129, 'Benwood meadows - 2': 131, 'Big Meadow - 1': 47, 'Big Meadow - 2': 48, 'Big Meadow - 3': 38, 'Big Meadow - 4': 39, 'Big Meadow - 5': 37, 'Big Meadow - 6': 40, 'Big meadow - 7': 126, 'Big Meadow Creek - lower': 521, 'Big Meadow Creek - upper': 491, 'Big Meadow Creek - up

In [ ]:
import pprint

# Display the dictionary using pprint
pprint.pprint(lookup_dict)


### Code for Grading each parameter

In [3]:
# Grading for each parameter 
#Defining Grade for Bank Stability based on Erosiondf[percent_unstable]
def categorize_erosion(Percent_Unstable):
    if 0 <= Percent_Unstable < 5:
        return 'A'
    elif 5 <= Percent_Unstable < 20:
        return 'B'
    elif 20 <= Percent_Unstable < 50:
        return 'C'
    else:
        return 'D'
    
#Scoring based off of grading - check this
def score_indicator(Rating):
    if  Rating == 'A':
        return '12'
    elif Rating == 'B':
        return '9'
    elif Rating == 'C':
        return '6'
    else:
        return '3'

#Define Grade for Incision based off of incisino ratio

def categorize_incision(bankfull_ratio):
    if 0 <= bankfull_ratio < 1.2:
        return 'A'
    elif 1.2 <= bankfull_ratio < 1.6:
        return 'B'
    elif 1.6 <= bankfull_ratio < 2.1:
        return 'C'
    else:
        return 'D'

#Define Grade for Bioassessment Score
def categorize_csci(biotic_integrity):
     if   biotic_integrity > 0.92:
        return 'A'
     elif 0.79 < biotic_integrity <= 0.92:
        return 'B'
     elif 0.62 < biotic_integrity <= 0.79:
        return 'C'
     else:
        return 'D'

#Define Priority List Level of Invasive Plant Species
    

# Process Data
def rate_invasive(priority):
    # Check if the SEZ has at least one large headcut
    if (priority[3] + priority[4] == 1):
        return 'B'  # Assign score D
    elif (priority[3] + priority[4] == 2) or priority[1] == 1 or priority[2] == 1:
        return 'C'  # Assign score C
    elif (priority[3] + priority[4] >= 3)or priority[1] >= 2 or priority[2] >= 2 or (priority[1] + priority[2] >= 2):  
       return 'B'  # Assign score B
    else:
        return 'A'  # Assign score A (no invasives of any priority are present)
     




#Define Size for Headcut based off of headcut size
##A = 0 headcut, B 1+small headcut
def categorize_headcut(headcutdepth):
    if pd.isnull(headcutdepth) or headcutdepth == 0:
        return np.nan
    elif 0.1 <= headcutdepth < 0.5:
        return 'small'
    elif 0.5 <= headcutdepth < 1:
        return 'medium'
    else:
        return 'large'

#define rating for headcut health per sez
def rate_headcut(row):
    # Check if the SEZ has at least one large headcut
    if row['large'] >= 1:
        return 'D'  # Assign score D
    elif row['medium'] >= 1:
        return 'C'  # Assign score C
    elif row['small'] >= 1:
       return 'B'  # Assign score B
    else:
        return 'A'  # Assign score A (no headcuts)




### Stream Erosion 

In [ ]:
#----------------------------------------------------------------#
#Get Data
#----------------------------------------------------------------##Convert feature class in gdb to Pandas DataFrame this is for REST Service
erosionfields = ['Assessment_Unit_Name', 'Shape.STLength()', 'Bank_Type', 'Survey_Date']
#erosiondf = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(erosiondata, erosionfields), columns=erosionfields)

# Initialize an empty list to store data
data = []

 # Iterate over the rows in the feature class and extract data
with arcpy.da.SearchCursor(erosiondata, erosionfields) as cursor:
    for row in cursor:
        data.append(row)

# Convert the data to a Pandas DataFrame
erosiondf = pd.DataFrame(data, columns=erosionfields)

# Replace NaN values in 'Assessment_Unit_Name' column with 'Skylandia SEZ'
#erosiondf['Assessment_Unit_Name'] = erosiondf['Assessment_Unit_Name'].fillna('Skylandia SEZ')
# Replace specific values in 'Assessment_Unit_Name' column
erosiondf['Assessment_Unit_Name'] = erosiondf['Assessment_Unit_Name'].replace({'Blackwood Creek - upper 2': 'Blackwood Creek - Upper 2', 'Taylor Creek marsh - 1': 'Taylor Creek marsh'})


#Add SEZ_ID column using lookup dictionary
#erosiondf['SEZ_ID'] = erosiondf['Assessment_Unit_Name'].map(lambda x: lookup_dict.get(x, {}).get('SEZ_ID', None))

#This code is for the excel look up dictionary
erosiondf['SEZ_ID'] = erosiondf['Assessment_Unit_Name'].map(lookup_dict)


# Fill NaN values with a specific value, such as 0
erosiondf['SEZ_ID'] = erosiondf['SEZ_ID'].fillna(0)

# Convert SEZ ID column to integer
erosiondf['SEZ_ID'] = erosiondf['SEZ_ID'].astype(int)

#erosiondf = erosiondf.merge(df, on='Assessment_Unit_Name', how='left')

#calculate year column 
erosiondf['Year'] = erosiondf['Survey_Date'].dt.year

# Replace 'both_banks' with 'Both Banks' in Bank_Type column
erosiondf['Bank_Type'] = erosiondf['Bank_Type'].replace(['both_banks', 'Both banks'], 'Both Banks' )
erosiondf['Bank_Type'] = erosiondf['Bank_Type'].replace(['one_bank', 'One bank'], 'One Bank')
erosiondf['Bank_Type'] = erosiondf['Bank_Type'].replace(['no_bank', 'No bank'], 'No Bank')

#----------------------------------------------------------------#
#Process Data
#----------------------------------------------------------------#

# Initialize variables
erosiondf['bank_multiplier'] = erosiondf['Bank_Type'].apply(lambda x: 2 if x == 'Both Banks' else (1 if x == 'One Bank' else 0))


# Calculate the product of 'Shape.STLength()' and 'bank_multiplier' to get the eroded banks per row
erosiondf['eroded_banks_per_row'] = erosiondf['Shape.STLength()'] * erosiondf['bank_multiplier']

# Group by Assessment_Unit_Name and year and sum the lengths of banks for each unit to get total banks assessed
erosiondf['banks_assessed_per_unit'] = erosiondf.groupby(['Assessment_Unit_Name', 'Year'])['Shape.STLength()'].transform('sum') * 2

# Group by Assessment_Unit_Name and sum the eroded banks per row for each unit
erosiondf['SEZ_total_eroded'] = erosiondf.groupby(['Assessment_Unit_Name', 'Year'])['eroded_banks_per_row'].transform('sum')

# Calculate percent unstable Bank Stability per Assessment Unit
erosiondf['Bank_Stability_Percent_Unstable'] = (erosiondf['SEZ_total_eroded'] / erosiondf['banks_assessed_per_unit']) * 100

#----------------------------------------------------------------#
#Grade, Score
#----------------------------------------------------------------#
erosiondf['Bank_Stability_Rating']=erosiondf['Bank_Stability_Percent_Unstable'].apply(categorize_erosion)
erosiondf['Bank_Stability_Score']= erosiondf['Bank_Stability_Rating'].apply(score_indicator)

erosiondf['Bank_Stability_Data_Source'] = 'TRPA'

erosiondf.head()

print(erosiondf)
#----------------------------------------------------------------#
#post ending dataframe to bank_stability called stage_bank_stability GDB location
#----------------------------------------------------------------#
# Define the name of the feature class
feature_class_name = 'bank_stability'

# Define the full path to the feature class
feature_class_path = stage_bank_stability 

#Field Mapping
field_mapping = {
    'Assessment_Unit_Name': 'Assessment_Unit_Name',
    'Year': 'Year',
    'Bank_Stability_Data_Source': 'Bank_Stability_Data_Source',
    'Bank_Stability_Percent_Unstable': 'Bank_Stability_Percent_Unstable',
    'Bank_Stability_Rating': 'Bank_Stability_Rating',
    'Bank_Stability_Score': 'Bank_Stability_Score',
    'SEZ_ID': 'SEZ_ID'
}

# Rename fields based on field mappings
bank_stabilitydf = erosiondf.rename(columns=field_mapping).drop(columns=[col for col in erosiondf.columns if col not in field_mapping])

readydf = bank_stabilitydf.groupby(['SEZ_ID', 'Year']).first().reset_index()

# Convert "Year" column to datetime format with year frequency
#readydf['Year'] = pd.to_datetime(readydf['Year'], format='%Y')

# Fix data type of Year so it writes to table
#readydf['Year'] = pd.to_datetime(readydf['Year'], format='%Y', errors='coerce')

# Setting the frequency to 'Y' for year
#readydf['Year'] = readydf['Year'].dt.to_period('Y')

print(readydf)

#Write dataframe to fc in SEZ_Data.GDB
# Convert DataFrame to a list of dictionaries
data = readydf.to_dict(orient='records')

# Get the field names from the field mapping
field_names = list(readydf.columns)

# Append data to existing table
#with arcpy.da.InsertCursor(stage_bank_stability, field_names) as cursor:
    for row in data:
        cursor.insertRow([row[field] for field in field_names])

print("Data appended to table successfully.")


#Write dataframe to sde.collect.bank.stability eventually, current code write it to GDB in SEZ_Data.GDB
# Set environment workspace to your SDE connection file
#arcpy.env.workspace = master_path

# Convert DataFrame to Feature Class
#output_feature_class = "ErosionUpdate"  # Name for the output feature class
#output_fc_path = os.path.join(arcpy.env.workspace, output_feature_class)

# Assuming your DataFrame is already converted to a feature class
# Replace "path_to_your_feature_class" with the actual path to your feature class
#arcpy.conversion.TableToTable("path_to_your_feature_class", arcpy.env.workspace, output_feature_class)

# Overwrite Feature Class in SDE
# Replace "path_to_your_dataframe" with the actual path to your DataFrame
#arcpy.management.CopyFeatures("path_to_your_dataframe", output_fc_path)


#print(ready_df)



In [ ]:
print(readydf.dtypes)

In [ ]:
# Convert DataFrame to CSV
#csv_path = os.path.join(working_folder, "Erosiondatamaster.csv")
#ready_df.to_csv(csv_path, index=False) 

### Channel Incision

In [ ]:
#----------------------------------------------------------------#
#Get Data-- all the data we need is in sde.collect.sde.survey.sde.sez_survey
#----------------------------------------------------------------##Convert feature class in gdb to Pandas DataFrame this is for REST Service
incisionfields = ['Assessment_Unit_Name', 'incision_ratio', 'survey_date']
#incisiondf = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(erosiondata, erosionfields), columns=erosionfields)

# Initialize an empty list to store data
data = []

 # Iterate over the rows in the feature class and extract data
with arcpy.da.SearchCursor(sezsurveytable, incisionfields) as cursor:
    for row in cursor:
        data.append(row)

# Convert the data to a Pandas DataFrame
incisiondf = pd.DataFrame(data, columns=incisionfields)






#----------------------------------------------------------------#
#Add correct info to dataframe
#----------------------------------------------------------------#
#use this until we fix the domain
incisiondf['Assessment_Unit_Name'] = incisiondf['Assessment_Unit_Name'].replace({'Blackwood Creek - upper 2': 'Blackwood Creek - Upper 2', 'Taylor Creek marsh - 1': 'Taylor Creek marsh'})
# Create a new column 'SEZ ID'
#This code is for the excel look up dictionary
incisiondf['SEZ_ID'] = incisiondf['Assessment_Unit_Name'].map(lookup_dict)


# Fill NaN values with a specific value, such as 0
incisiondf['SEZ_ID'] = incisiondf['SEZ_ID'].fillna(0)

# Convert SEZ ID column to integer
incisiondf['SEZ_ID'] = incisiondf['SEZ_ID'].astype(int)

#calculate year column 
incisiondf['Year'] = incisiondf['survey_date'].dt.year


#incisiondf['SEZ ID'] = np.nan

# Iterate through the rows in incisiondf
#for index, row in incisiondf.iterrows():
 #   parentglobalid = row['parentglobalid']
    
      # Check if the parent_global_id exists in the lookup dictionary
  #  if parentglobalid in lookup_dict:
        # If it exists, retrieve the corresponding SEZ ID and fill it into SEZ ID column
   #     corresponding_entry = lookup_dict[parentglobalid]
    #    assert row['GlobalID'] == parentglobalid, "ParentGlobalID does not match GlobalID in the lookup dictionary"
     #   incisiondf.at[index, 'SEZ_ID'] = corresponding_entry['SEZ_ID']

# Display the updated incisiondf
print(incisiondf)

#Add SEZ_ID column using lookup dictionary
#incisiondf['SEZ_ID'] = SEZIDdf['GlobalID'].map(lambda x: lookup_dict.get(x, {}).get('SEZ_ID', None))

# Fill NaN values with a specific value, such as 0
#incisiondf['SEZ_ID'] = incisiondf['SEZ_ID'].fillna(0)

# Convert SEZ ID column to integer
#incisiondf['SEZ_ID'] = incisiondf['SEZ_ID'].astype(int)

#incisiondf = incisiondf.merge(df, on='Assessment_Unit_Name', how='left')




#----------------------------------------------------------------#
#Grade, Score
#----------------------------------------------------------------#
incisiondf['Incision_Rating']=incisiondf['incision_ratio'].apply(categorize_incision)
incisiondf['Incision_Score']= incisiondf['Incision_Rating'].apply(score_indicator)

incisiondf['Incision_Data_Source'] = 'TRPA'

incisiondf.head()
#----------------------------------------------------------------#
#post ending dataframe to incision called stage_incision GDB location
#----------------------------------------------------------------#
# Define the name of the feature class
feature_class_name = 'incision'

# Define the full path to the feature class
feature_class_path = stage_incision 

#Field Mapping
field_mapping = {
    'Assessment_Unit_Name': 'Assessment_Unit_Name',
    'Year': 'Year',
    'Incision_Data_Source': 'Incision_Data_Source',
    'incision_ratio': 'Incision_Ratio',
    'Incision_Rating': 'Incision_Rating',
    'Incision_Score': 'Incision_Score',
    'SEZ_ID': 'SEZ_ID'
}

# Rename fields based on field mappings
incisionfinaldf = incisiondf.rename(columns=field_mapping).drop(columns=[col for col in incisiondf.columns if col not in field_mapping])

readydf = incisionfinaldf.groupby(['SEZ_ID', 'Year']).first().reset_index()

# Convert "Year" column to datetime format with year frequency
#readydf['Year'] = pd.to_datetime(readydf['Year'], format='%Y')

print(readydf)

#Write dataframe to fc in SEZ_Data.GDB
# Convert DataFrame to a list of dictionaries
data = readydf.to_dict(orient='records')

# Get the field names from the field mapping
field_names = list(readydf.columns)

# Append data to existing table
#with arcpy.da.InsertCursor(stage_incision, field_names) as cursor:
    for row in data:
        cursor.insertRow([row[field] for field in field_names])

print("Data appended to table successfully.")

### Invasive Species

In [ ]:
#Calfora Data points--dates only go to 2012?
# dates only go to 2012 hereCArest = "https://map.dfg.ca.gov/arcgis/rest/services/Project_BIOS_Public/q_BIOS_Public_pointslines07/MapServer/1"
#CArest = "https://map.dfg.ca.gov/arcgis/rest/services/Project_BIOS_Public/q_BIOS_Public_pointslines27/MapServer/0"
# Create a feature layer object
#cafeature_layer = FeatureLayer(CArest)
#Query caflora layer to that it only shows 2020-2023 or only county?
#start_year = 'January 1, 2020'
#end_year= 'December 31, 2023'
# Convert to datetime objects
#start_date = pd.to_datetime(start_year)
#end_date = pd.to_datetime(end_year)

#Query before or after??
#query = (CAsdf['Date_']>= start_date) & (CAsdf['Date_']<= end_date)

#CA_filtereddf = CAsdf.join[query].copy()

#### Code using REST Service- most likely will reuse this one

In [4]:

import pandas as pd
from arcgis.features import FeatureLayer
from utils import get_fs_data_spatial_query

# Define the USFS REST endpoint
usfsrest = "https://apps.fs.usda.gov/arcx/rest/services/EDW/EDW_InvasiveSpecies_01/MapServer/0"
where    = "FS_UNIT_ID = '0519'"

# Query the feature layer
sdfUSFS = get_fs_data_spatial_query(usfsrest, where)

In [5]:
#spatial join of sdf and sez master


# Create the spatially enabled DataFrame (sdf) for target feature SEZ assessment units
SEZsdf = pd.DataFrame.spatial.from_featureclass(SEZ_Master)
#spatial reference stuff
sdfUSFS.spatial.sr = SEZsdf.spatial.sr

#CAsdf.spatial.set_spatial_reference(SEZsdf.spatial.sr)
#perform spatial join
usfsdata = SEZsdf.spatial.join(sdfUSFS)

In [ ]:
print(invasivedf['plant_type'].unique())

In [25]:
#----------------------------------------
#SET UP DATA WRANGLE
#-----------------------------------------

#Path to external data usfs with rest service--This assumes rest service is up to date to 2023

# Paths to the feature classes
invasive19fc = os.path.join(invasiveplant19gdb, "Invasive_Species_2019")
invasive20fc = os.path.join(invasiveplant20gdb, "Invasive_Species_2020")
invasive22fc = os.path.join(invasiveplant22gdb, "Invasive_Species_2022")
invasive23fc = os.path.join(invasiveplant23gdb, "sez_invasive_plant")


#Use gdb because they have assessment unit name in them... 2022 i had to manually add in PRO
invasive23fields = ['Assessment_Unit', 'invasives_percent_cover','invasives_plant_type', 'Species_Level', 'invasive_type_other', 'created_date']
invasive22fields = ['Assessment_Unit_Name', 'Invasives_Plant_Type', 'Invasives_Percent_Cover', 'InvasiveType_Other', 'Survey_Date']
invasive20fields = ['Assessment_Unit_Name', 'Invasives_Plant_Type', 'Invasives_Percent_Cover',  'Other', 'Survey_Date']
invasive19fields = ['SITE_ID', 'SURVEY_DATE', 'INVASIVE_PLANT', 'PERCENT_COVER' ]
usfsfields = ['Assessment_Unit_Name', 'COMMON_NAME', 'SCIENTIFIC_NAME']

# Initialize an empty list to store data
data = []


# Function to read feature class into DataFrame
def feature_class_to_dataframe(feature_class, fields):
    data = [row for row in arcpy.da.SearchCursor(feature_class, fields)]
    return pd.DataFrame(data, columns=fields)

# Read feature classes into DataFrames
# Read feature classes into DataFrames
invasive19df = feature_class_to_dataframe(invasive19fc, invasive19fields)
invasive20df = feature_class_to_dataframe(invasive20fc, invasive20fields)
invasive22df = feature_class_to_dataframe(invasive22fc, invasive22fields)
invasive23df = feature_class_to_dataframe(invasive23fc, invasive23fields)
# Create a new DataFrame with only the specified columns
# Convert SpatialDataFrame to DataFrame

usfsdf = usfsdata[usfsfields]
#usfsdf = usfsdata.drop(columns='SHAPE')

print(usfsdf)
#usfs23df = feature_class_to_dataframe(usfsdata, usfsfields)


invasive19df['Source'] = 'TRPA'
invasive20df['Source'] = 'TRPA'
invasive22df['Source'] = 'TRPA'
invasive23df['Source'] = 'TRPA'
usfsdf['Source'] = 'USFS'

#usfsdf['Year'] = '2023'


# Rename fields
invasive19df.rename(columns={'SITE_ID': 'Assessment_Unit_Name', 'INVASIVE_PLANT': 'plant_type', 'PERCENT_COVER': 'percent_cover', 'SURVEY_DATE': 'created_date','Source':'Source'}, inplace=True)
invasive20df.rename(columns={'Assessment_Unit_Name': 'Assessment_Unit_Name', 'Invasives_Plant_Type': 'plant_type', 'Other': 'other', 'Invasives_Percent_Cover': 'percent_cover', 'Survey_Date': 'created_date', 'Source':'Source'}, inplace=True)
invasive22df.rename(columns={'Assessment_Unit': 'Assessment_Unit_Name', 'Invasives_Plant_Type': 'plant_type', 'InvasiveType_Other': 'other', 'Invasives_Percent_Cover': 'percent_cover', 'Survey_Date': 'created_date','Source':'Source'}, inplace=True)
invasive23df.rename(columns={'Assessment_Unit': 'Assessment_Unit_Name', 'invasives_percent_cover': 'percent_cover', 'invasives_plant_type': 'plant_type', 'invasive_type_other': 'other', 'created_date': 'created_date','Source':'Source'}, inplace=True)
usfsdf.rename(columns={'Assessment_Unit_Name': 'Assessment_Unit_Name', 'COMMON_NAME': 'plant_type', 'Source':'Source'}, inplace=True)
#Remove null plant types for usfs data
usfsdf = usfsdf[~usfsdf['plant_type'].isna()]


# Concatenate DataFrames
invasivedf = pd.concat([usfsdf, invasive19df, invasive20df, invasive22df, invasive23df], ignore_index=True)
# Display merged DataFrame
print(invasivedf)


#Define SEZ ID based on Assessment_Unit_Name for QA on SEZ Name 
invasivedf['SEZ_ID'] = invasivedf['Assessment_Unit_Name'].map(lookup_dict)

# Fill NaN values with a specific value, such as 0
invasivedf['SEZ_ID'] = invasivedf['SEZ_ID'].fillna(0)

# Convert SEZ ID column to integer
#invasivedf['SEZ_ID'] = invasivedf['SEZ_ID'].astype(int)
invasivedf['Assessment_Unit_Name'] = invasivedf['Assessment_Unit_Name'].replace({'Blackwood Creek - upper 2': 'Blackwood Creek - Upper 2', 'Taylor Creek marsh - 1': 'Taylor Creek marsh', 'Sky Meadows meadow - 1': 'Sky meadows'})
# Remove rows where data source is USFS and plant type is NaN
#invasivedf = invasivedf[~((invasivedf['Source'] == 'USFS') & invasivedf['plant_type'].isna())]

# Set 'Year' column based on data source

invasivedf.loc[invasivedf['Source'] == 'USFS', 'Year'] = '2023'
invasivedf.loc[invasivedf['Source'] == 'TRPA', 'Year'] = invasivedf['created_date'].dt.year

#invasivedf.reset_index(drop=True, inplace=True)
invasivedf.reset_index(drop=True, inplace=True)

#invasivedf = invasivedf.drop_duplicates()


                             Assessment_Unit_Name   COMMON_NAME  \
0    Saxon Creek meadows - above Fountain Place 1  bull thistle   
1                         Burke Creek meadows - 2    cheatgrass   
2                             Burke Creek - upper    cheatgrass   
3       Edgewood Creek tributary - 2 - headwaters    cheatgrass   
4                             Burke Creek - upper    cheatgrass   
..                                            ...           ...   
914          Edgewood Creek tributary - 3 - upper  bull thistle   
915          Edgewood Creek tributary - 3 - upper  bull thistle   
916          Edgewood Creek tributary - 3 - upper   oxeye daisy   
917          Edgewood Creek tributary - 2 - upper  bull thistle   
918          Edgewood Creek tributary - 2 - upper  bull thistle   

          SCIENTIFIC_NAME  
0         Cirsium vulgare  
1         Bromus tectorum  
2         Bromus tectorum  
3         Bromus tectorum  
4         Bromus tectorum  
..                    ...  

C:\Users\snewsome\AppData\Local\Temp\ipykernel_7176\2686383337.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usfsdf['Source'] = 'USFS'
C:\Users\snewsome\AppData\Local\Temp\ipykernel_7176\2686383337.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usfsdf.rename(columns={'Assessment_Unit_Name': 'Assessment_Unit_Name', 'COMMON_NAME': 'plant_type', 'Source':'Source'}, inplace=True)


In [ ]:
invasivedf_values_unique = invasivedf.values.flatten()
is_unique = len(invasivedf_values_unique) == len(set(invasivedf_values_unique))
print(is_unique)

# Check for duplicate rows
duplicate_rows = invasivedf[invasivedf.duplicated(keep=False)]

# Display duplicate rows
print(duplicate_rows)


In [28]:
#---------------------------#
    #Prep Plant_type Data
#---------------------------#
#Make a dataframe to capture 'other' plants in trpa data and then add it to invasive df
other_plants_df = invasivedf[['Source', 'Year', 'SEZ_ID', 'Assessment_Unit_Name', 'Year', 'other']].copy()

#Get rid of Null values
other_plants_df = other_plants_df[~other_plants_df['other'].isna()]

#invasivedf.reset_index(drop=True, inplace=True)
other_plants_df.reset_index(drop=True, inplace=True)


#Rename 'other to plant_type
other_plants_df.rename(columns={'other': 'plant_type'}, inplace=True)

#invasivedf = invasivedf.drop_duplicates()


# Concatenate other_plants_df with invasivedf JUST DO THIS MANUALLY 
#invasivedf = pd.concat([invasivedf, other_plants_df], ignore_index=True)

#Append
#invasivesdf=invasivedf.append(other_plants_df)
#Concatenate the new DataFrame with the existing invasivedf
#invasivedf = pd.concat([invasivedf, other_plants_df], ignore_index=True)


# Replace various representations of null values with 'none'
null_representations = ['<null>', '<Null>', '', 'NA', 'N/A', 'nan', 'NaN', 'None', 'NULL', None]
invasivedf['plant_type'] = invasivedf['plant_type'].replace(null_representations, 'none')

# Split plant types by comma and create new rows
invasivedf['plant_type'] = invasivedf['plant_type'].str.split(pat=',')
invasivedf = invasivedf.explode('plant_type')

# Capitalize the first word and replace underscores with spaces
invasivedf['plant_type'] = invasivedf['plant_type'].str.split('_').str[0].str.capitalize() + ' ' + invasivedf['plant_type'].str.split('_').str[1:].str.join(' ')

# Remove spaces after the words and capitalize the first word while replacing underscores with spaces
invasivedf['plant_type'] = invasivedf['plant_type'].astype(str)


# Remove any spaces at the beginning or end of a word
invasivedf['plant_type'] = invasivedf['plant_type'].str.strip()


#---------------------#
#These replacements are so the same weed with different common name doesn't get counted twice toward score MAY NEED TO ADD MORE DESCREPENCIES in coming years if more pop up or find more elopquent solution
#---------------------#
# Replace 'Common mullein' with 'Wooly mullein' in the 'plant_type' column 
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Common mullein', 'Wooly mullein')
# Replace 'Nodding plumeless thistle' with 'Musk thistle' in the 'plant_type' column
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Nodding plumeless thistle', 'Musk thistle')
# Replace 'Nodding plumeless thistle' with 'Musk thistle' in the 'plant_type' column
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Field bindweed', 'Common bindweed')
# Replace 'Nodding plumeless thistle' with 'Musk thistle' in the 'plant_type' column
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Common st. johnswort', 'Klamathweed')
# Replace 'Broadleaf and leaved' with 'Perennial' in the 'plant_type' column
# Define the replacements dictionary
replacements = {'Broadleaf Pepperweed': 'Perennial pepperweed', 'Broadleaved pepperweed': 'Perennial pepperweed'}

# Replace values in the 'plant_type' column using the dictionary
invasivedf['plant_type'] = invasivedf['plant_type'].replace(replacements)

#Replace Sulphur Cinquefoil with sulfur
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Sulphur cinquefoil', 'Sulfur cinquefoil')
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Sweetclover', 'White sweetclover')
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Reed canarygrass', 'Reed canary grass')
#Replace
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Salt cedar', 'Tamarisk')
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Butter and eggs', 'Yellow toadflax')
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Canada cottonthistle', 'Canada thistle')
# Replace empty strings or other placeholders with NaN
#invasivedf['plant_type'] = invasivedf['plant_type'].replace('', np.nan)

# Drop duplicates based on 'Assessment_Unit_Name' and 'Year' in the remaining DataFrame
invasivedf = invasivedf.drop_duplicates(subset=['Assessment_Unit_Name', 'Year', 'plant_type'], keep='first')


grouped_df = invasivedf.groupby(['Assessment_Unit_Name', 'Year'])['plant_type']

# Aggregate the plant types into one column separated by commas
combined_plant_types = grouped_df.apply(lambda x: ', '.join(x)).reset_index(name='all_plant_types')





In [24]:
for group, data in invasivedf.groupby(['Assessment_Unit_Name', 'Year']):
    print(group)
    print(data)
    print('-------------------------')

('Angora Creek - tributary', 2022.0)
          Assessment_Unit_Name plant_type SCIENTIFIC_NAME Source  \
1134  Angora Creek - tributary       None            <NA>   TRPA   

            created_date  percent_cover other  Species_Level  SEZ_ID    Year  
1134 2022-08-10 12:00:00            0.0  None            NaN   519.0  2022.0  
-------------------------
('Angora Creek - tributary', 2023.0)
          Assessment_Unit_Name    plant_type SCIENTIFIC_NAME Source  \
1213  Angora Creek - tributary  Bull thistle            <NA>   TRPA   

            created_date  percent_cover other  Species_Level  SEZ_ID    Year  
1213 2023-05-25 15:19:53            5.0  None            3.0   519.0  2023.0  
-------------------------
('Angora Creek - tributary', '2023')
        Assessment_Unit_Name    plant_type  SCIENTIFIC_NAME Source  \
81  Angora Creek - tributary  Bull thistle  Cirsium vulgare   USFS   

   created_date  percent_cover other  Species_Level  SEZ_ID  Year  
81          NaT            NaN  

In [29]:
#----------------------------------------------------#
#Create Look up dictionary to assign priority group for each plant and then use it
#----------------------------------------------------#

# Step 1: Read the Excel file into a DataFrame
csv_data = pd.read_csv(r"F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Invasives Priority lookup.csv") 

#Define Empty look up dataframe
Invasives_lookup = {}

key = 'Common'
values = ['Scientific', 'Priority'] 

Invasives_lookup= csv_data.set_index(key)[values].to_dict(orient='index')

# Define a custom function to map plant types to priorities
def map_priority(plant_type):
    if pd.isnull(plant_type):
        return 'None' # Return NaN for NaN values
    else:
        # Extract the priority from the dictionary, or return 'Unknown' if not found
        plant_info = Invasives_lookup.get(plant_type)
        if plant_info:
            return plant_info['Priority']
        else:
            return 'Unknown'


# Create a new column 'Priority' based on the mapping from the dictionary
invasivedf['Priority'] = invasivedf['plant_type'].map(map_priority)

print(invasivedf.columns)
#invasivedf['Priority'] = pd.to_numeric(invasivedf['Priority'], errors='coerce')

# group by assessment unit and year and summarize the priority level of plants in each unit
invasive_summary = invasivedf.groupby(['Assessment_Unit_Name', 'Year', 'Priority','Source'], dropna=False).size().reset_index(name='Count')

invasive_summary_priority = invasive_summary.pivot_table(index=['Assessment_Unit_Name', 'Year','Source'], columns='Priority', values='Count', fill_value=0)


# Reset the index to flatten the DataFrame
invasive_summary_priority.reset_index(inplace=True)

Index(['Assessment_Unit_Name', 'plant_type', 'SCIENTIFIC_NAME', 'Source',
       'created_date', 'percent_cover', 'other', 'Species_Level', 'SEZ_ID',
       'Year', 'Priority'],
      dtype='object')


In [30]:
#----------------------------------------------------------------#
#Grade, Score, # of Headcuts
#----------------------------------------------------------------#

# Process Data
def rate_invasive(priority):
    if (priority[3] + priority[4] == 1):
        return 'B'  # Assign score D
    elif (priority[3] + priority[4] == 2) or priority[1] == 1 or priority[2] == 1:
        return 'C'  # Assign score C
    elif (priority[3] + priority[4] >= 3)or priority[1] >= 2 or priority[2] >= 2 or (priority[1] + priority[2] >= 2):  
       return 'B'  # Assign score B
    else:
        return 'A'  # Assign score A (no invasives of any priority are present)

# Apply the rating function to the summary DataFrame
invasive_summary_priority['Invasives_Rating'] = invasive_summary_priority[[1, 2, 3, 4]].apply(rate_invasive, axis=1)

#Calculate the score for the sez
invasive_summary_priority['Invasives_Score']= invasive_summary_priority['Invasives_Rating'].apply(score_indicator) 

#Calculate total number of invasives per sez per year
invasive_summary_priority['Number_of_Invasives']= invasive_summary_priority[[1, 2, 3, 4]].sum(axis=1)

#Define SEZ ID based on Assessment_Unit_Name for QA on SEZ Name 
invasive_summary_priority['SEZ_ID'] = invasive_summary_priority['Assessment_Unit_Name'].map(lookup_dict)

invasive_summary_priority['all_plants']= combined_plant_types['all_plant_types']

In [32]:
#----------------------------------------------------------------#
#Prep and post ending dataframe to invasives table in SEZ_Data.GDB
#----------------------------------------------------------------#
#Field Mapping
field_mapping = {
    'Assessment_Unit_Name': 'Assessment_Unit_Name',
    'Year': 'Year',
    'Data_Sources': 'Invasives_Data_Source',
    'Number_of_Invasives': 'Invasives_Number_of_Invasives',
    'Invasives_Rating': 'Invasives_Rating',
    'Invasives_Score': 'Invasives_Scores',
    'SEZ_ID': 'SEZ_ID',
    'percent_cover': 'Invasives_Percent_Cover',
    'all_plants': 'Invasives_Plant_Type',
}

# Rename fields based on field mappings
readydf = invasive_summary_priority.rename(columns=field_mapping).drop(columns=[col for col in invasive_summary_priority.columns if col not in field_mapping])

readydf['SEZ_ID'] = readydf['Assessment_Unit_Name'].map(lookup_dict)

# Convert "Year" column to datetime format with year frequency
#readydf['Year'] = pd.to_datetime(readydf['Year'], format='%Y')

#print(readydf)

# Convert DataFrame to a list of dictionaries
#data = readydf.to_dict(orient='records')

# Get the field names from the field mapping
#field_names = list(readydf.columns)

# Append data to existing table
#with arcpy.da.InsertCursor(stage_headcuts, field_names) as cursor:
 #   for row in data:
  #      cursor.insertRow([row[field] for field in field_names])

#print("Data appended to table successfully.")

#### Invasive with gdb and usfs pre joined layer

In [ ]:
#Invasive Species


#Path to external data usfs 
usfsdata = os.path.join(master_path, "usfsinvasivesez24")
#Path to external data Calflora aka State Park Data
#calfloradata = os.path.join(master_path, "")

# Paths to the feature classes
invasive19fc = os.path.join(invasiveplant19gdb, "Invasive_Species_2019")
invasive20fc = os.path.join(invasiveplant20gdb, "Invasive_Species_2020")
invasive22fc = os.path.join(invasiveplant22gdb, "Invasive_Species_2022")
invasive23fc = os.path.join(invasiveplant23gdb, "sez_invasive_plant")


#Use gdb because they have assessment unit name in them... 2022 i had to manually add in PRO
invasive23fields = ['Assessment_Unit', 'invasives_percent_cover','invasives_plant_type', 'Species_Level', 'invasive_type_other', 'created_date']
invasive22fields = ['Assessment_Unit_Name', 'Invasives_Plant_Type', 'Invasives_Percent_Cover', 'InvasiveType_Other', 'Survey_Date']
invasive20fields = ['Assessment_Unit_Name', 'Invasives_Plant_Type', 'Invasives_Percent_Cover',  'Other', 'Survey_Date']
invasive19fields = ['SITE_ID', 'SURVEY_DATE', 'INVASIVE_PLANT', 'PERCENT_COVER' ]
usfsfields = ['Assessment_Unit_Name', 'PLANT_COMM', 'SCIENTIFIC', 'DATE_COLLE', 'Eradicated']

# Initialize an empty list to store data
data = []


# Function to read feature class into DataFrame
def feature_class_to_dataframe(feature_class, fields):
    data = [row for row in arcpy.da.SearchCursor(feature_class, fields)]
    return pd.DataFrame(data, columns=fields)

# Read feature classes into DataFrames
# Read feature classes into DataFrames
invasive19df = feature_class_to_dataframe(invasive19fc, invasive19fields)
invasive20df = feature_class_to_dataframe(invasive20fc, invasive20fields)
invasive22df = feature_class_to_dataframe(invasive22fc, invasive22fields)
invasive23df = feature_class_to_dataframe(invasive23fc, invasive23fields)
usfs23df = feature_class_to_dataframe(usfsdata, usfsfields)


invasive19df['Source'] = 'TRPA'
invasive20df['Source'] = 'TRPA'
invasive22df['Source'] = 'TRPA'
invasive23df['Source'] = 'TRPA'
usfs23df['Source'] = 'USFS'

# Rename fields
invasive19df.rename(columns={'SITE_ID': 'Assessment_Unit_Name', 'INVASIVE_PLANT': 'plant_type', 'PERCENT_COVER': 'percent_cover', 'SURVEY_DATE': 'created_date','Source':'Source'}, inplace=True)
invasive20df.rename(columns={'Assessment_Unit_Name': 'Assessment_Unit_Name', 'Invasives_Plant_Type': 'plant_type', 'Other': 'other', 'Invasives_Percent_Cover': 'percent_cover', 'Survey_Date': 'created_date', 'Source':'Source1'}, inplace=True)
invasive22df.rename(columns={'Assessment_Unit': 'Assessment_Unit_Name', 'Invasives_Plant_Type': 'plant_type', 'InvasiveType_Other': 'other', 'Invasives_Percent_Cover': 'percent_cover', 'Survey_Date': 'created_date','Source':'Source1'}, inplace=True)
invasive23df.rename(columns={'Assessment_Unit': 'Assessment_Unit_Name', 'invasives_percent_cover': 'percent_cover', 'invasives_plant_type': 'plant_type', 'invasive_type_other': 'other', 'created_date': 'created_date','Source':'Source1'}, inplace=True)
usfs23df.rename(columns={'Assessment_Unit_Name': 'Assessment_Unit_Name', 'PLANT_COMM':'plant_type', 'DATE_COLLE':'created_date', 'Source':'Source2'}, inplace=True)


# Concatenate DataFrames
invasivedf = pd.concat([usfs23df, invasive19df, invasive20df, invasive22df, invasive23df], ignore_index=True)

# Display merged DataFrame
print(invasivedf)

#Define SEZ ID based on Assessment_Unit_Name
invasivedf['SEZ_ID'] = invasivedf['Assessment_Unit_Name'].map(lookup_dict)

# Fill NaN values with a specific value, such as 0
invasivedf['SEZ_ID'] = invasivedf['SEZ_ID'].fillna(0)

# Convert SEZ ID column to integer
invasivedf['SEZ_ID'] = invasivedf['SEZ_ID'].astype(int)
invasivedf['Assessment_Unit_Name'] = invasivedf['Assessment_Unit_Name'].replace({'Blackwood Creek - upper 2': 'Blackwood Creek - Upper 2', 'Taylor Creek marsh - 1': 'Taylor Creek marsh', 'Sky Meadows meadow - 1': 'Sky meadows'})

#calculate year column 
invasivedf['Year'] = invasivedf['created_date'].dt.year


#---------------------------#
    #Prep Data
#---------------------------#
# Replace 'other' or 'Other' in 'plant_type' column with values from 'other' column
invasivedf['plant_type'] = invasivedf.apply(lambda row: row['other'] if pd.notna(row['plant_type']) and row['plant_type'].lower() in ['other', 'Other'] else row['plant_type'], axis=1)

# Drop the 'other' column
invasivedf.drop(columns=['other'], inplace=True)

# Function to separate plant types and create new rows
def separate_species(df):
    # Split plant types by comma and create new rows
    df['plant_type'] = df['plant_type'].str.split(',')
    df = df.explode('plant_type')
    return df

# Apply the function to your DataFrame
invasivedf = separate_species(invasivedf)

# Remove rows where data source is USFS and plant type is NaN
invasivedf = invasivedf[~((invasivedf['Source'] == 'USFS') & invasivedf['plant_type'].isna())]

# Capitalize the first word and replace underscores with spaces
invasivedf['plant_type'] = invasivedf['plant_type'].str.split('_').str[0].str.capitalize() + ' ' + invasivedf['plant_type'].str.split('_').str[1:].str.join(' ')

# Remove spaces after the words and capitalize the first word while replacing underscores with spaces
invasivedf['plant_type'] = invasivedf['plant_type'].astype(str)


# Remove any spaces at the beginning or end of a word
invasivedf['plant_type'] = invasivedf['plant_type'].str.strip()


# Remove rows where data source is USFS and plant type is NaN
invasivedf = invasivedf[~((invasivedf['Source'] == 'USFS') & invasivedf['plant_type'].isna())]
#Remove Eradicated 

# Filter out rows where 'eradicated' column is 'Yes'
invasivedf = invasivedf[invasivedf['Eradicated'] != 'Yes']

#These replacements are so the same weed with different common name doesn't get counted twice toward score MAY NEED TO ADD MORE DESCREPENCIES in coming years if more pop up or find more elopquent solution
# Replace 'Common mullein' with 'Wooly mullein' in the 'plant_type' column 
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Common mullein', 'Wooly mullein')
# Replace 'Nodding plumeless thistle' with 'Musk thistle' in the 'plant_type' column
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Nodding plumeless thistle', 'Musk thistle')
# Replace 'Nodding plumeless thistle' with 'Musk thistle' in the 'plant_type' column
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Field bindweed', 'Common bindweed')
# Replace 'Nodding plumeless thistle' with 'Musk thistle' in the 'plant_type' column
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Common st. johnswort', 'Klamathweed')
# Replace 'Broadleaf and leaved' with 'Perennial' in the 'plant_type' column
# Define the replacements dictionary
replacements = {'Broadleaf Pepperweed': 'Perennial pepperweed', 'Broadleaved pepperweed': 'Perennial pepperweed'}

# Replace values in the 'plant_type' column using the dictionary
invasivedf['plant_type'] = invasivedf['plant_type'].replace(replacements)

#Replace Sulphur Cinquefoil with sulfur
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Sulphur cinquefoil', 'Sulfur cinquefoil')
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Sweetclover', 'White sweetclover')

#Replace
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Salt cedar', 'Tamarisk')

# Now, drop duplicates based on the specified subset of columns
invasivedf = invasivedf.drop_duplicates(subset=['SEZ_ID', 'Year', 'plant_type'])

# Reset index if needed
invasivedf.reset_index(drop=True, inplace=True)


# Remove duplicates based on SEZ, Year, and plant_type
invasivedf = invasivedf.drop_duplicates(subset=['SEZ_ID', 'Year', 'plant_type'])
#----------------------------------------------------#
#Create Look up dictionary to assign priority group for each plant and then use it
#----------------------------------------------------#

# Step 1: Read the Excel file into a DataFrame
csv_data = pd.read_csv(r"F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Invasives Priority lookup.csv") 

#Define Empty look up dataframe
Invasives_lookup = {}

key = 'Common'
values = ['Scientific', 'Priority'] 

Invasives_lookup= csv_data.set_index(key)[values].to_dict(orient='index')

# Define a custom function to map plant types to priorities
def map_priority(plant_type):
    if pd.isnull(plant_type):
        return np.nan  # Return NaN for NaN values
    else:
        # Extract the priority from the dictionary, or return 'Unknown' if not found
        plant_info = Invasives_lookup.get(plant_type)
        if plant_info:
            return plant_info['Priority']
        else:
            return 'Unknown'

#Create a new column [Scientific based on look up dictionary
#invasivedf['Scientific']=invasivedf['plant_type'].map(map_priority)

# Create a new column 'Priority' based on the mapping from the dictionary
invasivedf['Priority'] = invasivedf['plant_type'].map(map_priority)

#invasivedf['Priority'] = pd.to_numeric(invasivedf['Priority'], errors='coerce')
# group by assessment unit and year and summarize the priority level of plants in each unit
invasive_summary = invasivedf.groupby(['Assessment_Unit_Name', 'Year', 'Priority']).size().reset_index(name='Count')

invasive_summary_priority2 = invasive_summary.pivot_table(index=['Assessment_Unit_Name', 'Year'], columns='Priority', values='Count', fill_value=0)

# Reset the index to flatten the DataFrame
invasive_summary_priority2.reset_index(inplace=True)

#invasive_summary_priority['Source'] = invasivedf['Source']
#----------------------------------------------------------------#
#Grade, Score, # of Headcuts
#----------------------------------------------------------------#

# Process Data
def rate_invasive(priority):
    # Check if the SEZ has at least one large headcut
    if (priority[3] + priority[4] == 1):
        return 'B'  # Assign score D
    elif (priority[3] + priority[4] == 2) or priority[1] == 1 or priority[2] == 1:
        return 'C'  # Assign score C
    elif (priority[3] + priority[4] >= 3)or priority[1] >= 2 or priority[2] >= 2 or (priority[1] + priority[2] >= 2):  
       return 'B'  # Assign score B
    else:
        return 'A'  # Assign score A (no invasives of any priority are present)

# Apply the rating function to the summary DataFrame
invasive_summary_priority2['Invasives_Rating'] = invasive_summary_priority2.apply(rate_invasive, axis=1)

#Calculate total number of invasives per sez per year
invasive_summary_priority2['Number_of_Invasives']= invasive_summary_priority2[[1, 2, 3, 4,'Unknown']].sum(axis=1)

#Calculate the score for the sez
invasive_summary_priority2['Invasives_Score']= invasive_summary_priority2['Invasives_Rating'].apply(score_indicator)    

# make a columns in invasive summary that totals up percent cover per sez/year
invasive_summary_priority2['Invasives_Percent_Cover'] = invasivedf.groupby(['SEZ_ID', 'Year'])['percent_cover'].sum().reset_index(drop=True)

# combine the source column so that it shows all data sources that contributed to the data
# Group invasivedf by 'SEZ_ID' and 'Year' and concatenate 'Data Source 1' values
#data_source_1_combined = invasivedf.groupby(['SEZ_ID', 'Year'])['Source 1'].apply(lambda x: ', '.join(x.dropna())).reset_index()

# Group invasivedf by 'SEZ_ID' and 'Year' and concatenate 'Data Source 2' values
#data_source_2_combined = invasivedf.groupby(['SEZ_ID', 'Year'])['Source 2'].apply(lambda x: ', '.join(x.dropna())).reset_index()

# Merge data_source_1_combined and data_source_2_combined on 'SEZ_ID' and 'Year'
#merged_data_sources = pd.merge(data_source_1_combined, data_source_2_combined, on=['SEZ_ID', 'Year'], how='outer')

# Combine 'Data Source 1' and 'Data Source 2' values with a comma separator
#merged_data_sources['Data_Sources'] = merged_data_sources.apply(lambda row: ', '.join(filter(None, [row['Source 1'], row['Source 2']])), axis=1)

# Drop the individual 'Data Source 1' and 'Data Source 2' columns
#merged_data_sources.drop(columns=['Source 1', 'Source 2'], inplace=True)

# Merge merged_data_sources with invasive_summary_priority on 'SEZ_ID' and 'Year'
#invasive_summary_priority = pd.merge(invasive_summary_priority, merged_data_sources, on=['SEZ_ID', 'Year'], how='left')

#invasive_summary_priority['Data_Sources']= invasivedfinvasivedf.groupby(['SED_ID', 'Year'])[Data Source 1 ] merge with DataSource 1 separate with comma if there are both 
#----------------------------------------------------------------#
#post ending dataframe to incision called stage_incision GDB location
#----------------------------------------------------------------#


#Field Mapping
field_mapping = {
    'Assessment_Unit_Name': 'Assessment_Unit_Name',
    'Year': 'Year',
    'Data_Sources': 'Invasives_Data_Source',
    'Number_of_Invasives': 'Invasives_Number_of_Invasives',
    'Invasives_Rating': 'Invasives_Rating',
    'Invasives_Score': 'Invasives_Scores',
    'SEZ_ID': 'SEZ_ID',
    'percent_cover': 'Invasives_Percent_Cover',
    'plant_type': 'Invasives_Plant_Type',
}

# Rename fields based on field mappings
#readydf = invasive_summary_priority.rename(columns=field_mapping).drop(columns=[col for col in headcut_summary_sml.columns if col not in field_mapping])

#readydf['SEZ_ID'] = readydf['Assessment_Unit_Name'].map(lookup_dict)

# Convert "Year" column to datetime format with year frequency
#readydf['Year'] = pd.to_datetime(readydf['Year'], format='%Y')

#print(readydf)

# Convert DataFrame to a list of dictionaries
#data = readydf.to_dict(orient='records')

# Get the field names from the field mapping
#field_names = list(readydf.columns)

# Append data to existing table
#with arcpy.da.InsertCursor(stage_headcuts, field_names) as cursor:
 #   for row in data:
  #      cursor.insertRow([row[field] for field in field_names])

#print("Data appended to table successfully.")


In [ ]:
print(invasive_summary_priority.columns)


In [ ]:
plant_type= 'Cheatgrass'
plant_info = Invasives_lookup.get(plant_type)



In [ ]:
print(Invasives_lookup)

In [ ]:
#GDB SET UP DONT USE
# Step 1: Read the Excel file into a DataFrame
csv_data = pd.read_csv(r"F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Invasives Priority lookup.csv") 

#Define Empty look up dataframe
Invasives_lookup = {}

key = 'Common'
values = ['Scientific', 'Priority'] 

Invasives_lookup= csv_data.set_index(key)[values].to_dict(orient='index')
#Extra fields in case I need them but cleaned up in the code up top.. if i don't need these fields by the end of processing delete this code
#Use gdb because they have assessment unit name in them... 2022 i had to manually add in PRO
invasive23fields = ['Assessment_Unit', 'invasives_percent_cover', 'invasives_number_of_species','invasives_plant_type', 'Species_Level', 'invasive_type_other', 'created_date']
invasive22fields = ['Assessment_Unit_Name', 'Invasives_Plant_Type', 'Invasives_Percent_Cover', 'Invasives_Number_of_Species', 'InvasiveType_Other', 'Survey_Date']
invasive20fields = ['Assessment_Unit_Name', 'Invasives_Plant_Type', 'Invasives_Percent_Cover', 'Invasives_Number_of_Species', 'Other', 'Survey_Date']
invasive19fields = ['SITE_ID', 'TOTAL_SPECIES', 'SURVEY_DATE', 'INVASIVE_PLANT', 'PERCENT_COVER' ]
usfsfields = ['Assessment_Unit_Name', 'PLANT_COMM', 'SCIENTIFIC', 'DATE_COLLE', 'Eradicated']

# Initialize an empty list to store data
data = []


# Function to read feature class into DataFrame
def feature_class_to_dataframe(feature_class, fields):
    data = [row for row in arcpy.da.SearchCursor(feature_class, fields)]
    return pd.DataFrame(data, columns=fields)

# Read feature classes into DataFrames
# Read feature classes into DataFrames
invasive19df = feature_class_to_dataframe(invasive19fc, invasive19fields)
invasive20df = feature_class_to_dataframe(invasive20fc, invasive20fields)
invasive22df = feature_class_to_dataframe(invasive22fc, invasive22fields)
invasive23df = feature_class_to_dataframe(invasive23fc, invasive23fields)
usfs23df = feature_class_to_dataframe(usfsdata, usfsfields)



# Rename fields
invasive19df.rename(columns={'SITE_ID': 'Assessment_Unit_Name', 'INVASIVE_PLANT': 'plant_type', 'TOTAL_SPECIES': 'total',  'PERCENT_COVER': 'percent_cover', 'SURVEY_DATE': 'created_date','Source':'Source'}, inplace=True)
invasive20df.rename(columns={'Assessment_Unit_Name': 'Assessment_Unit_Name', 'Invasives_Plant_Type': 'plant_type', 'Other': 'other', 'Invasives_Percent_Cover': 'percent_cover','Invasives_Number_of_Species': 'total', 'Survey_Date': 'created_date', 'Source':'Source'}, inplace=True)
invasive22df.rename(columns={'Assessment_Unit': 'Assessment_Unit_Name', 'Invasives_Plant_Type': 'plant_type', 'InvasiveType_Other': 'other', 'Invasives_Percent_Cover': 'percent_cover','Invasives_Number_of_Species': 'total', 'Survey_Date': 'created_date','Source':'Source'}, inplace=True)
invasive23df.rename(columns={'Assessment_Unit': 'Assessment_Unit_Name', 'invasives_percent_cover': 'percent_cover', 'invasives_number_of_species': 'total', 'invasives_plant_type': 'plant_type', 'invasive_type_other': 'other', 'created_date': 'created_date','Source':'Source'}, inplace=True)
usfs23df.rename(columns={'Assessment_Unit_Name': 'Assessment_Unit_Name', 'PLANT_COMM':'plant_type', 'DATE_COLLE':'created_date', 'Source':'Source'}, inplace=True)

In [ ]:
# Check the data type of 'plant_type' column in the DataFrame
print("Data type of 'plant_type' column in DataFrame:", invasivedf['plant_type'].dtype)

# Check the data type of values in the lookup dictionary
for key, value in Invasives_lookup.items():
    print("Data type of value for key", key, "in lookup dictionary:", type(value))


In [ ]:
#DONT USE

# Step 1: Read the Excel file into a DataFrame
csv_data = pd.read_csv(r"F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Invasives Priority lookup.csv")

# Define an empty dictionary for the lookup
invasives_lookup = {}

# Print a message before the assignment
print("Before assignment:", invasives_lookup)

# Iterate over rows in the CSV data
for index, row in csv_data.iterrows():
    common_name = row['Common']
    scientific_name = row['Scientific']
    priority = int(row['Priority'])
    
    # Assign values to the dictionary
    invasives_lookup[common_name] = {'Scientific': scientific_name, 'Priority': priority}

# Print a message after the assignment
print("After assignment:", invasives_lookup)


### HEadcuts


In [ ]:
#Headcuts
#--------------------------------#
#Get Data from GDB's with assessment unit name 
#--------------------------------#

# Paths to the feature classes
headcut19fc = os.path.join(headcut19gdb, "Stream_Headcut_2019")
headcut20fc = os.path.join(headcut20gdb, "Stream_Headcut_2020")
headcut22fc = os.path.join(headcut22gdb, "Stream_Headcut_2022")
headcut23fc = os.path.join(headcut23gdb, "sez_stream_headcut")


#Use gdb because they have assessment unit name in them... 2022 i had to manually add in PRO
headcut23fields = ['Assessment_Unit', 'headcut_depth', 'created_date']
headcut22fields = ['Assessment_Unit', 'Headcut_Depth', 'synced_date']
headcut20fields = ['Assessment_Unit_Name', 'Headcut_Depth','Survey_Date']
headcut19fields = ['SITE_NAME', 'HEADCUT_DEPTH', 'SURVEY_DATE' ]
# Initialize an empty list to store data
data = []


# Function to read feature class into DataFrame
def feature_class_to_dataframe(feature_class, fields):
    data = [row for row in arcpy.da.SearchCursor(feature_class, fields)]
    return pd.DataFrame(data, columns=fields)

# Read feature classes into DataFrames
# Read feature classes into DataFrames
headcut19df = feature_class_to_dataframe(headcut19fc, headcut19fields)
headcut20df = feature_class_to_dataframe(headcut20fc, headcut20fields)
headcut22df = feature_class_to_dataframe(headcut22fc, headcut22fields)
headcut23df = feature_class_to_dataframe(headcut23fc, headcut23fields)

# Rename fields
headcut19df.rename(columns={'SITE_NAME': 'Assessment_Unit_Name', 'HEADCUT_DEPTH': 'headcut_depth', 'SURVEY_DATE': 'created_date'}, inplace=True)
headcut20df.rename(columns={'Assessment_Unit_Name': 'Assessment_Unit_Name', 'Headcut_Depth': 'headcut_depth', 'Survey_Date': 'created_date'}, inplace=True)
headcut22df.rename(columns={'Assessment_Unit': 'Assessment_Unit_Name', 'Headcut_Depth': 'headcut_depth', 'synced_date': 'created_date'}, inplace=True)
headcut23df.rename(columns={'Assessment_Unit': 'Assessment_Unit_Name', 'headcut_depth': 'headcut_depth', 'created_date': 'created_date'}, inplace=True)

# Concatenate DataFrames
headcutdf = pd.concat([headcut19df, headcut20df, headcut22df, headcut23df], ignore_index=True)

# Display merged DataFrame
print(headcutdf)

#hopefully this is just one time


#--------------------------------#
#Get Data from exported table where Assessment Unit is joined---parentglobalids don't match up for years before 2023. need to redo sde.collect 2019-2022 data append.... 
#--------------------------------#

#--------------------------------#
#Get Data from SDE.Collect...instead of GDB
#--------------------------------#
#----------------------------------------------------------------#
#Add correct info to dataframe
#----------------------------------------------------------------#
# Create a new column 'SEZ ID'
#This code is for the excel look up dictionary---ADD last? this is goo dfor QA on assessment unit names because 0's tell you the name is wrong
headcutdf['SEZ_ID'] = headcutdf['Assessment_Unit_Name'].map(lookup_dict)


# Fill NaN values with a specific value, such as 0
headcutdf['SEZ_ID'] = headcutdf['SEZ_ID'].fillna(0)

# Convert SEZ ID column to integer
headcutdf['SEZ_ID'] = headcutdf['SEZ_ID'].astype(int)
headcutdf['Assessment_Unit_Name'] = headcutdf['Assessment_Unit_Name'].replace({'Blackwood Creek - upper 2': 'Blackwood Creek - Upper 2', 'Taylor Creek marsh - 1': 'Taylor Creek marsh', 'Sky meadows - 1': 'Sky Meadows'})

#calculate year column 
headcutdf['Year'] = headcutdf['created_date'].dt.year

#----------------------------------------------#
# Process Data
#----------------------------------------------#
# assign small, medium, large to headcut
headcutdf['Headcut_Size']=headcutdf['headcut_depth'].apply(categorize_headcut)


# Group by 'SEZ_ID', 'Year', and 'Headcut_Size', and count the number of occurrences for each group
headcut_summary = headcutdf.groupby(['Assessment_Unit_Name', 'Year', 'Headcut_Size']).size().reset_index(name='Count')

print(type(headcut_summary))
# Pivot the table to have 'Headcut_Size' categories as columns
headcut_summary_sml = headcut_summary.pivot_table(index=['Assessment_Unit_Name', 'Year'], columns='Headcut_Size', values='Count', fill_value=0)

# Reset the index to flatten the DataFrame
headcut_summary_sml.reset_index(inplace=True)





# Display the summarized DataFrame
print(headcut_summary_sml)

#----------------------------------------------------------------#
#Grade, Score, # of Headcuts
#----------------------------------------------------------------#
    
    # Process Data
def rate_headcut(row):
    # Check if the SEZ has at least one large headcut
    if row['large'] >= 1:
        return 'D'  # Assign score D
    elif row['medium'] >= 1:
        return 'C'  # Assign score C
    elif row['small'] >= 1:
       return 'B'  # Assign score B
    else:
        return 'A'  # Assign score A (no headcuts)

# Apply the rating function to the summary DataFrame
headcut_summary_sml['Headcuts_Rating'] = headcut_summary_sml.apply(rate_headcut, axis=1)

#Calculate total number of headcuts per sez per year
headcut_summary_sml['Number_of_Headcuts']= headcut_summary_sml[['large', 'medium', 'small']].sum(axis=1)

#Calculate the score for the sez
headcut_summary_sml['Headcuts_Score']= headcut_summary_sml['Headcuts_Rating'].apply(score_indicator)

#Add Datasource
headcut_summary_sml['Headcuts_Data_Source'] = 'TRPA'


#----------------------------------------------------------------#
#post ending dataframe to incision called stage_incision GDB location
#----------------------------------------------------------------#


#Field Mapping
field_mapping = {
    'Assessment_Unit_Name': 'Assessment_Unit_Name',
    'Year': 'Year',
    'Headcuts_Data_Source': 'Headcuts_Data_Source',
    'Number_of_Headcuts': 'Number_of_Headcuts',
    'Headcuts_Rating': 'Headcuts_Rating',
    'Headcuts_Score': 'Headcuts_Score',
    'SEZ_ID': 'SEZ_ID',
    'small': 'small',
    'medium': 'medium',
    'large': 'large'
}

# Rename fields based on field mappings
readydf = headcut_summary_sml.rename(columns=field_mapping).drop(columns=[col for col in headcut_summary_sml.columns if col not in field_mapping])

readydf['SEZ_ID'] = readydf['Assessment_Unit_Name'].map(lookup_dict)

# Convert "Year" column to datetime format with year frequency
#readydf['Year'] = pd.to_datetime(readydf['Year'], format='%Y')

print(readydf)

# Convert DataFrame to a list of dictionaries
data = readydf.to_dict(orient='records')

# Get the field names from the field mapping
field_names = list(readydf.columns)

# Append data to existing table
with arcpy.da.InsertCursor(stage_headcuts, field_names) as cursor:
    for row in data:
        cursor.insertRow([row[field] for field in field_names])

print("Data appended to table successfully.")



In [ ]:
# Bring in External Data from USFS and Calflora

In [ ]:
#--------------------------------#
#Get Data from external sources and spatially join to our data
#--------------------------------#


#Calfora Data
CArest = "https://map.dfg.ca.gov/arcgis/rest/services/Project_BIOS_Public/q_BIOS_Public_pointslines07/MapServer/1"
#CAfc = os.path.join(CArest, "Non-Native and Invasive Plants - Calflora")
CAsdf = pd.DataFrame.spatial.from_layer(CArest)
# Delete the existing feature layer if it exists

CA_fc = arcpy.MakeFeatureLayer_management(CArest, "Calfora_invasives")


usfsrest= "https://apps.fs.usda.gov/arcx/rest/services/EDW/EDW_InvasiveSpecies_01/MapServer/0"
#usfsfc =os.path.join(usfsrest, "Current Invasive Plant Locations") 
#usfssdf = pd.DataFrame.spatial.from_layer(usfsrest)
if arcpy.Exists("usfs_layer"):
    arcpy.Delete_management("usfs_layer")
usfs_fc = arcpy.MakeFeatureLayer_management(usfsrest, "usfs_layer")
#Spatially join usfs layers to see if any of the points or polygons intersect with our sez's123

# Set the workspace environment
arcpy.env.workspace = r"F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Data Management 2023.gdb"

# Define the target and join layers
target_feature = SEZ_Master #"AssessmentUnit_Master"
CAjoin_feature = CA_fc
USFSjoin_feature = usfs_fc

# Define the output feature class
out_feature_class = r"F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Data Management 2023.gdb\externalinvasivesez"

# Define the fields you want to keep from both layers
#field_mappings = arcpy.FieldMappings()
#field_mappings.addTable(target_features)
#field_mappings.addTable(join_features)

# Keep only specific fields from the target layer
#keep_fields_target = ["Assessment_Unit_Name"]
#for field in field_mappings.fields:
 #   if field.name not in keep_fields_target:
  #      field_mappings.removeFieldMap(field_mappings.findFieldMapIndex(field.name))

#Keep only specific fields from the calfora join layer
#keep_fields_CA = ["CName", "SName", "Source", "Obs_Date"]
#for field in field_mappings.fields:
 #   if field.name not in keep_fields_CA:
  #      field_mappings.removeFieldMap(field_mappings.findFieldMapIndex(field.name))

# Keep only specific fields from the usfs join_features
#keep_fields_USFS = ["SCIENTIFIC_NAME", "COMMON_NAME", "DATE_COLLECTED"]
#for field in field_mappings.fields:
 #   if field.name not in keep_fields_USFS:
  #      field_mappings.removeFieldMap(field_mappings.findFieldMapIndex(field.name))


# Perform spatial join with specified field mappings
arcpy.analysis.SpatialJoin(
    target_features=SEZ_Master,
    join_features=[CAjoin_feature],
    out_feature_class=out_feature_class,
    join_operation="JOIN_ONE_TO_MANY",
    join_type="KEEP_ALL"
    #field_mapping=field_mappings
)

#Create Dataframe for spatially joined data
# Define the fields you want to retrieve from the output feature class
fields = ["Assessment_Unit_Name", "CName", "SName", "Source", "Obs_Date", "SCIENTIFIC_NAME", "COMMON_NAME", "DATE_COLLECTED"]

# Create a DataFrame to store the data
data = []

# Use SearchCursor to read data from the output feature class
with arcpy.da.SearchCursor(out_feature_class, fields) as cursor:
    for row in cursor:
        data.append(row)

# Convert the list of tuples into a pandas DataFrame
df = pd.DataFrame(data, columns=fields)

# Display the DataFrame
print(df)


### Bioassessment/ Biotic Integrity

In [ ]:
#Bioassessment scores- get all stream data into sde.Stream first... then look at Biotic Integrit Data Source to find which stream was used to evaluate each meadow.. this will help with percent of stream miles I believe

#----------------------------------------------------------------#
#Get Data-- all the data we need is in sde.collect.sde.survey.sde.sez_survey
#----------------------------------------------------------------##Convert feature class in gdb to Pandas DataFrame this is for REST Service

#Create SEDF setup
#streamdata is the path to the feature class in sde
#stream_fl = FeatureLayer(streamdata)
#Get CSCI scores for streams
#streamfields = ['SITE_NAME', 'COUNT_VALUE', 'YEAR_OF_COUNT', 'LATITUDE', 'LONGITUDE']

# Convert the data to a Pandas DataFrame
streamsdf = GeoAccessor.from_layer(streamdata)

#streamsedf = GeoAccessor.from_xy(streamdata, x_column='LONGITUDE', y_column='LATITUDE')

if the layer contains Riverine?
#spatial join this layer to asessment unit master layer
#ASsessment unit master layer is called SEZ_Master

#spatial join of sez to stream data
#bioticsdf = SEZsdf.spatial.join(streamsdf, how='inner')
#----------------------------------------------------------------#
#Add correct info to dataframe
#----------------------------------------------------------------#
#use this until we fix the domain
#bioticsdf['Assessment_Unit_Name'] = bioticsdf['Assessment_Unit_Name'].replace({'Blackwood Creek - upper 2': 'Blackwood Creek - Upper 2', 'Taylor Creek marsh - 1': 'Taylor Creek marsh'})
# Create a new column 'SEZ ID'
#This code is for the csv look up dictionary
#bioticsdf['SEZ_ID'] = bioticsdf['Assessment_Unit_Name'].map(lookup_dict)


# Fill NaN values with a specific value, such as 0
#bioticsdf['SEZ_ID'] = bioticsdf['SEZ_ID'].fillna(0)

# Convert SEZ ID column to integer
#bioticsdf['SEZ_ID'] = bioticsdf['SEZ_ID'].astype(int)




In [ ]:
#Conifer Encroachment

In [ ]:
#Aquatic Organism /Fish Passage

##### Code to make it a spatially enabled dataframe

In [ ]:
#code to conver to spatially enable geodataframe
# Create a new DataFrame Final_df with columns based on the field mapping
#df = df.rename(columns=field_mapping)

# Convert DataFrame to Spatially Enabled DataFrame
#sedf = GeoAccessor.from_xy(df, x_column='LONGITUDE', y_column='LATITUDE')

# Convert the SEDF to a feature class without sanitizing columns
#sedf.spatial.to_featureclass(location=os.path.join(workspace, 'PlanAreaNoise_23_Staging'), sanitize_columns=False)